## Titanic (test) - Machine Learning from Disaster

In [1]:
from utils import *

In [2]:
data = pd.read_csv('dataset/train.csv')
test = pd.read_csv('dataset/test.csv')
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [3]:
print(test.shape)
test.dtypes

(418, 11)


PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [4]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [5]:
test.duplicated().sum()

0

In [6]:
test, passenger_ids = preprocess_data(test, is_train=False)
print(test.duplicated().sum())
test.isnull().sum()

3


Pclass        0
Sex           0
Age           0
Fare          0
Embarked      0
TicketCode    0
FamilySize    0
IsAlone       0
dtype: int64

In [7]:
test.shape

(418, 8)

In [8]:
test.shape

(418, 8)

In [9]:
passenger_ids = passenger_ids.loc[test.index]
passenger_ids

0       892
1       893
2       894
3       895
4       896
       ... 
413    1305
414    1306
415    1307
416    1308
417    1309
Name: PassengerId, Length: 418, dtype: int64

In [10]:
numerical_cols = test.select_dtypes(include=['number']).columns
for col in numerical_cols:
    outliers = detect_outliers_iqr(test, col)
    print(f"Outliers in {col}: {len(outliers)}")

Outliers in Pclass: 2
Outliers in Age: 2
Outliers in Fare: 2
Outliers in FamilySize: 2
Outliers in IsAlone: 2


In [11]:
test = encode_feature(test)
test.head()

,Pclass,Sex,Age,Fare,TicketCode,FamilySize,IsAlone,Embarked_Q,Embarked_S
0,3,0,34.5,7.8292,330911,1,1,1,0
1,3,1,47.0,7.0000,363272,2,0,0,1
2,2,0,62.0,9.6875,240276,1,1,1,0
3,3,0,27.0,8.6625,315154,1,1,0,1
4,3,1,22.0,12.2875,3101298,3,0,0,1


In [12]:
scaler = joblib.load('scaler.pkl')
cols_to_scale = ['Age', 'Fare', 'FamilySize']
test[cols_to_scale] = scaler.transform(test[cols_to_scale])

# Ensure test columns match training columns
expected_columns = joblib.load('train_columns.pkl')
test = test.reindex(columns=expected_columns, fill_value=0) 

c:\Users\dbwjd\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator RobustScaler from version 1.6.1 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [13]:
model = joblib.load('SVC.pkl')
predictions = model.predict(test)

submission = pd.DataFrame({
    'PassengerId': passenger_ids,  # PassengerId
    'Survived': predictions
})

submission.to_csv('submission.csv', index=False)
print("Submission file saved as Submission.csv")

Submission file saved as Submission.csv


c:\Users\dbwjd\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.6.1 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [14]:
submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
